In [1]:
import pyxdf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from warnings import warn
import math
import ECGComplexity
import neurokit2 as nk

import os
import glob

## Export ECG data from xdf file

In [40]:
os.chdir(r'C:\Users\sruth\OneDrive - University of Illinois at Chicago\Documents\MATLAB\6. HIL\ECG_processing\Energy adaptation optimization\arl testing Sep 27 2024\User preferred\sub-P001\ses-S001\eeg')  # Folder path to the xdf file

print(os.getcwd())

filename = "sub-P001_ses-S001_task-Default_run-001_eeg.xdf" # File name

data, header = pyxdf.load_xdf(filename)

C:\Users\sruth\OneDrive - University of Illinois at Chicago\Documents\MATLAB\6. HIL\ECG_processing\Energy adaptation optimization\arl testing Sep 27 2024\User preferred\sub-P001\ses-S001\eeg


In [41]:
for data_streams in data:
    
    print(data_streams['info']['name'])
    
    if data_streams['info']['name']==['polar ECG 2']:
        ecg_data=data_streams['time_series'] # this is the ecg data
        ecg_time_stamps=data_streams['time_stamps']
        
    if data_streams['info']['name']==['polar accel']:
        accel_data=data_streams['time_series'] # this is the acceleration data
        accel_time_stamps=data_streams['time_stamps']

    if data_streams['info']['name']==['TestInf']:
        condition_data = data_streams['time_series'] # this is the condition data (e.g. "first parameter", "second parameter", etc)
        condition_time_stamps = data_streams['time_stamps']
    
    if data_streams['info']['name']==['Change_parm']:
        parm_data = data_streams['time_series'] 
        parm_time_stamps = data_streams['time_stamps']

['polar ECG 2']
['polar accel']
['Polar_symmetry']
['Met_cost']


In [42]:
ecg_data

array([[209.],
       [212.],
       [212.],
       ...,
       [274.],
       [288.],
       [308.]], dtype=float32)

In [43]:
ecg_time_stamps

array([769201.09526009, 769201.10294565, 769201.11063121, ...,
       769442.88295812, 769442.89064368, 769442.89832924])

In [44]:
# check time duration
x = ecg_time_stamps[-1] - ecg_time_stamps[0]
x

241.80306914064568

In [45]:
# Save raw ECG data 
df_ecg = pd.DataFrame(ecg_data, columns = ["ecg"])
df_ecg["ecg_time_stamps"] = ecg_time_stamps
df_ecg.to_excel("P0_ecg.xlsx") # Save file with appropriate filename

In [54]:
# # Code to select data of interest (i.e., between specific time stamps) from Excel file 

# os.chdir(r'C:\Users\sruth\OneDrive - University of Illinois at Chicago\Documents\MATLAB\6. HIL\ECG_processing\ETC optimization\VV_hip_exo_2')  # Folder path

# df = pd.read_excel("P01_ecg_hip_exo_NW.xlsx", index_col=0) # index column is not read

# df_new = df[(df['ecg_time_stamps'].between(327490.82-120, 327490.82))]  # last 120 seconds

# # check time duration
# t=df_new["ecg_time_stamps"].iloc[-1]-df_new["ecg_time_stamps"].iloc[0]
# t

# # Convert to Excel
# df_new.to_excel('P01_ecg_hip_exo_NW.xlsx')

# Extract RR intervals and HRV indices - Single file processing

In [2]:
# function to clean signal, extract R peaks in two formats - (1) array indices (2) RR intervals in milliseconds 

def ECG_process(ecg_signal, srate):
    # clean signal
    ECG_signal_clean = nk.ecg_clean(ecg_signal, sampling_rate=srate, method='neurokit')
    
    # Extract R peaks
    R_peaks=nk.ecg_peaks(ECG_signal_clean, sampling_rate=srate, method='pantompkins1985')
    
    # To access indices of R peaks
    peaks_idx=R_peaks[1]['ECG_R_Peaks']    
    
    # To obtain RR intervals in milliseconds from indices
    peaks_ms = get_RR(peaks_idx, srate)
    
    # To access R peaks as a vector of 0's and 1's. 
    # peaks_bin = R_peaks[0]

    return peaks_idx, peaks_ms

In [3]:
# function to get RR intervals in milliseconds

def get_RR(peaks_idx, srate):
    
    # Get successive differences (RR intervals)
    res = []
    res = [peaks_idx[i + 1] - peaks_idx[i] for i in range(len(peaks_idx) - 1)]
    peaks = res
    
    # Convert to milliseconds
    peaks = np.asarray(peaks[1:], dtype=np.float32)
    peaks_ms = np.round(peaks/srate*1000,0) # in Hz sampling rate
    
    return peaks_ms

In [4]:
# Function to calculate the frequency of occurrence of patterns sets of different complexities as per ETC

from collections import Counter

def ETC_distributions(peaks_ms):
    # Given array (RR intervals)
    l = ECGComplexity.symbolic_sequence_difference(peaks_ms)
   
    # Window size and overlap
    window_size = 8
    overlap = 7
    step = window_size - overlap
   
    moving_window_ETC = []
   
    # Extract moving window of 8 elements with an overlap of 7 elements and calculate its ETC
    for i in range(0, len(l)-window_size+1, step):
        subsequence = l[i:i+window_size]
        res = ECGComplexity.ETC(subsequence, "None")/(window_size-1)
        res = np.round(res, 4)
        moving_window_ETC.append(res)
   
    # Calculate the frequency distribution of the ETC values
    frequency_distribution = Counter(moving_window_ETC)
   
    # Access the frequency distribution values only after it has been computed
    ETC_L1 = frequency_distribution.get(0.1429, 0)  # Use a default of 0 if the key is not found
    ETC_L2 = frequency_distribution.get(0.4286, 0)
    ETC_L3 = frequency_distribution.get(0.5714, 0)
    ETC_L4 = frequency_distribution.get(0.7143, 0)
    ETC_L5 = frequency_distribution.get(0.8571, 0)
   
    # Calculate the frequency of occurrence of complex patterns (ETC > 0.7)
    ETC_L4L5 = ETC_L4 + ETC_L5
   
    return ETC_L4L5

In [5]:
# Function to calculate the frequency of occurrence of patterns sets of different complexities as per ETC, with normalization

def ETC_distributions_norm(peaks_ms):
    # Given array (RR intervals)
    l = ECGComplexity.symbolic_sequence_difference(peaks_ms)
   
    # Window size and overlap
    window_size = 8
    overlap = 7
    step = window_size - overlap
   
    moving_window_ETC = []
   
    # Extract moving window of 8 elements with an overlap of 7 elements and calculate its ETC
    for i in range(0, len(l)-window_size+1, step):
        subsequence = l[i:i+window_size]
        res = ECGComplexity.ETC(subsequence, "None") / (window_size-1)
        res = np.round(res, 4)
        moving_window_ETC.append(res)
   
    # Calculate the frequency distribution of the ETC values
    frequency_distribution = Counter(moving_window_ETC)
 
    # Get ETC levels with default value 0 if the key is not found
    ETC_L1 = frequency_distribution.get(0.1429, 0)
    ETC_L2 = frequency_distribution.get(0.4286, 0)
    ETC_L3 = frequency_distribution.get(0.5714, 0)
    ETC_L4 = frequency_distribution.get(0.7143, 0)
    ETC_L5 = frequency_distribution.get(0.8571, 0)
   
    # Avoid division by zero by checking if the denominator is zero
    total = sum([ETC_L1, ETC_L2, ETC_L3, ETC_L4, ETC_L5])
    if total == 0:
        return 0
   
    # Calculate the frequency of occurrence of complex patterns (ETC > 0.7)
    ETC_L4L5_norm = (ETC_L4 + ETC_L5) / total
   
    return ETC_L4L5_norm

In [60]:
# Change folder location

os.chdir(r'C:\Users\sruth\OneDrive - University of Illinois at Chicago\Documents\MATLAB\6. HIL\ECG_processing\Energy adaptation optimization\arl testing Sep 27 2024\User preferred\sub-P001\ses-S001\eeg')  # Folder path

In [62]:
# Import the ECG data
df_ECG = pd.read_excel("P0_ecg.xlsx")

# Specify sampling rate
srate = 130  # Hz

# Extract R peaks
ecg = df_ECG["ecg"].to_numpy()
[peaks_idx, peaks_ms] = ECG_process(ecg, srate)

### Code for extracting data after first 2 minutes ###
peaks_ms_cumsum = np.cumsum(peaks_ms)

# Find the index where the cumulative time exceeds 2 minutes (120,000 ms)
cut_off_index = np.where(peaks_ms_cumsum > (120 * 1000))[0][0]

# Extract the last two minutes of data for peaks_ms and peaks_idx
peaks_ms_trunc = peaks_ms[cut_off_index:]
peaks_idx_trunc = peaks_idx[cut_off_index:]

# Re-asssign values for further processing
peaks_ms = peaks_ms_trunc
peaks_idx = peaks_idx_trunc

# Check the duration (in ms) to ensure it's around 2 minutes (120,000 ms)
duration = np.cumsum(peaks_ms_trunc)[-1] - np.cumsum(peaks_ms_trunc)[0]
duration

# Optionally, convert numpy arrays to dataframes for saving to Excel
# df_peaks_ms = pd.DataFrame(peaks_ms)
# df_peaks_idx = pd.DataFrame(peaks_idx)

# Save R peaks and corresponding indices in Excel files
# df_peaks_ms.to_excel('P01_ecg_peaks_ms.xlsx')
# df_peaks_idx.to_excel('P01_ecg_peaks_idx.xlsx')

120588.0

In [52]:
# Extract Heart rate variability indices and store them in a new Excel file

hrv_indices = nk.hrv(peaks_idx, sampling_rate = srate, show = False)

# Length of RR intervals
hrv_indices["RR_len"] = len(peaks_ms)

# Calculate heart rate (in bpm)
hrv_indices["heart_rate"] = 60000/hrv_indices["HRV_MeanNN"] # since Mean_NN is in millisec

# Calculate complexity features
hrv_indices["ETC_diff"] = ECGComplexity.ETC(peaks_ms, "difference")
hrv_indices["ETC_thres"] = ECGComplexity.ETC(peaks_ms, "threshold")
hrv_indices["ETC_perc"] = ECGComplexity.ETC(peaks_ms, "percentile")

# Calculate frequency of occurrence of complex pattern sets
hrv_indices["ETC_L4L5"] = ETC_distributions(peaks_ms)
hrv_indices["ETC_L4L5_norm"] = ETC_distributions_norm(peaks_ms)

# Calculate symbolic dynamics features
[P0V_diff, P1V_diff, P2V_diff] = ECGComplexity.symbolic_dynamics(peaks_ms,"difference")
hrv_indices["0V_diff"] = P0V_diff
hrv_indices["1V_diff"] = P1V_diff
hrv_indices["2V_diff"] = P2V_diff

[P0V_thres, P1V_thres, P2V_thres] = ECGComplexity.symbolic_dynamics(peaks_ms,"threshold")
hrv_indices["0V_thres"] = P0V_thres
hrv_indices["1V_thres"] = P1V_thres
hrv_indices["2V_thres"] = P2V_thres

[P0V_perc, P1V_perc, P2V_perc] = ECGComplexity.symbolic_dynamics(peaks_ms,"percentile")
hrv_indices["0V_perc"] = P0V_perc
hrv_indices["1V_perc"] = P1V_perc
hrv_indices["2V_perc"] = P2V_perc

# Save all indices to an Excel file
hrv_indices.to_excel(f'P0_HRV_indices.xlsx')